##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
<td>
<a target="_blank" href="https://www.tensorflow.org/alpha/tutorials/sequences/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
</td><td>
<a target="_blank"  href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/sequences/text_generation.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank"  href="https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/sequences/text_generation.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

## Setup

### Import TensorFlow and other libraries

In [91]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

import numpy as np
import os
import time

### Download the  dataset

Change the following line to run this code on your own data.

# 天龍八部 dateset download here(自己抓的 目前只有前十幾章)--> https://drive.google.com/open?id=1NOCD5IkDEohFk0fh0Ii6Ki9oIT-1CjdJ

In [0]:
#from google.colab import files
#uploaded = files.upload()

In [94]:
ls

brother.txt  dragoneight.txt  sample_data/  training_checkpoints/


In [0]:
# rm -r 'brother.txt'

In [0]:
#path_to_file = tf.keras.utils.get_file('dragoneight.txt', '~./dragoneight.txt')

### Read the data

First, look in the text:

In [97]:
# Read, then decode for py2 compat.
text = open('dragoneight.txt', 'rb').read().decode(encoding='utf-8')
#text = open(path_to_file, 'rb').read().decode(encoding='unicode')
#text = open('brother.txt', 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 336691 characters


In [98]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿第一章　青衫磊落險峰行
青光閃動，一柄青鋼劍倏地刺出，指向在年漢子左肩，使劍少年不等招用老，腕抖劍斜，劍鋒已削向那漢子右頸。那中年漢子劍擋格，錚的一聲響，雙劍相擊，嗡嗡作聲，震聲未絕，雙劍劍光霍霍，已拆了三招，中年漢子長劍猛地擊落，直砍少年頂門。那少年避向右側，左手劍訣一引，青鋼劍疾刺那漢子大腿。 
　　兩人劍法迅捷，全力相搏。 
　　練武廳東坐著二人。上首是個四十左右的中年道姑，鐵青著臉，嘴唇緊閉。下首是個五十餘歲的老者，右手捻著長鬚，神情甚是得意。兩人的座位相距一丈有餘，身後各站著二


In [99]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

3445 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [101]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\x0c':   1,
  '\r':   2,
  ' ' :   3,
  '"' :   4,
  '(' :   5,
  ')' :   6,
  '*' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  ':' :  11,
  '?' :  12,
  '…' :  13,
  '●' :  14,
  '\u3000':  15,
  '、' :  16,
  '。' :  17,
  '「' :  18,
  '」' :  19,
  ...
}


In [102]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\ufeff第一章\u3000青衫磊落險峰行\r' ---- characters mapped to int ---- > [3434 2221   22 2209   15 3235 2687 2128 2577 3197  876 2678    2]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text. 

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right. 

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [103]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

﻿
第
一
章
　


The `batch` method lets us easily convert these individual characters to sequences of the desired size. 

In [104]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\ufeff第一章\u3000青衫磊落險峰行\r\n青光閃動，一柄青鋼劍倏地刺出，指向在年漢子左肩，使劍少年不等招用老，腕抖劍斜，劍鋒已削向那漢子右頸。那中年漢子劍擋格，錚的一聲響，雙劍相擊，嗡嗡作聲，震聲未絕，雙劍劍光霍霍，'
'已拆了三招，中年漢子長劍猛地擊落，直砍少年頂門。那少年避向右側，左手劍訣一引，青鋼劍疾刺那漢子大腿。 \r\n\u3000\u3000兩人劍法迅捷，全力相搏。 \r\n\u3000\u3000練武廳東坐著二人。上首是個四十左右的中年道姑，鐵青著臉，嘴'
'唇緊閉。下首是個五十餘歲的老者，右手捻著長鬚，神情甚是得意。兩人的座位相距一丈有餘，身後各站著二十餘名男女弟子。西邊一排椅子上坐著十餘位賓客。東西雙方的目光都集注於場中二人的角鬥。 \r\n\u3000\u3000眼見那少年與'
'中年漢子已拆到七十餘招，劍招越來越緊，兀自未分勝敗。突然中年漢子一劍揮出，用力猛了，身子微微一幌，似欲摔跌。西邊賓客中一個身穿青衫的年輕男子忍不住「嗤」的一聲笑。他隨即知道失態，忙伸手按住了口。 \r\n\u3000'
'\u3000便在這時，場中少年左手呼一掌拍出，擊向那漢子後心，那漢子向前跨出一步避開，手中長劍驀地圈轉，喝一聲：「著！」那少年左腿已然中劍，腿下一個踉蹌，長劍在地下一撐，站直身子待欲再鬥，那中年漢子已還劍入鞘，笑'


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [106]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeff第一章\u3000青衫磊落險峰行\r\n青光閃動，一柄青鋼劍倏地刺出，指向在年漢子左肩，使劍少年不等招用老，腕抖劍斜，劍鋒已削向那漢子右頸。那中年漢子劍擋格，錚的一聲響，雙劍相擊，嗡嗡作聲，震聲未絕，雙劍劍光霍霍'
Target data: '第一章\u3000青衫磊落險峰行\r\n青光閃動，一柄青鋼劍倏地刺出，指向在年漢子左肩，使劍少年不等招用老，腕抖劍斜，劍鋒已削向那漢子右頸。那中年漢子劍擋格，錚的一聲響，雙劍相擊，嗡嗡作聲，震聲未絕，雙劍劍光霍霍，'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [107]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 3434 ('\ufeff')
  expected output: 2221 ('第')
Step    1
  input: 2221 ('第')
  expected output: 22 ('一')
Step    2
  input: 22 ('一')
  expected output: 2209 ('章')
Step    3
  input: 2209 ('章')
  expected output: 15 ('\u3000')
Step    4
  input: 15 ('\u3000')
  expected output: 3235 ('青')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [108]:
# Batch size 
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 512

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units, 
                        return_sequences=True, 
                        stateful=True, 
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [111]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

W0410 19:10:02.867451 139812515600256 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f28315f46d8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next character:

![A drawing of the data passing through the model](https://tensorflow.org/tutorials/sequences/images/text_generation_training.png)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [112]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 3445) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length: 

In [113]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (128, None, 512)          1763840   
_________________________________________________________________
unified_lstm_2 (UnifiedLSTM) (128, None, 1024)         6295552   
_________________________________________________________________
dense_2 (Dense)              (128, None, 3445)         3531125   
Total params: 11,590,517
Trainable params: 11,590,517
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary. 

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [115]:
sampled_indices

array([3067,  603,  792,  941, 1962, 1160,  950, 2388,  708, 1076, 3097,
       1247, 1736, 3394,  575, 1915, 2574, 1317, 2944, 1202, 1920,  424,
       2918, 1486, 2296, 1544, 2433, 2786,  509, 1585, 2497,  617, 2267,
        437, 1892, 1147, 1330, 2070, 1487, 1563, 1950, 2055,  500, 3193,
       1618,  532, 1496, 3148,  613, 1416,   61,  933, 3051,  761, 1220,
       2904, 3137, 1639,  489, 2437,  429, 1360, 1788, 3261, 1095, 2855,
       2178, 3052, 1522, 3308, 3285,  481, 2816, 2722, 2861, 2512, 2873,
       1595, 3118,  266, 1040,  891,  899, 1040, 1103, 1485, 2247,  849,
       2997,  404, 3046, 2558, 1645, 3277, 2318,  771, 2344, 2996, 1060,
        396])

Decode these to see the text predicted by this untrained model:

In [116]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '惱恨表哥。」她鼓著勇氣說了這幾句話，但一出口，心中便怦怦亂跳，自驚怎地如此大膽，竟敢出言衝撞母親。 \r\n\u3000\u3000王夫人眼光如冷電，在女兒臉上掃了幾下，半晌不語，跟著便閉上了眼睛。王語嫣大氣也不敢透一口，不'

Next Char Predictions: 
 '酬因宏店瑚戚庸翩妍悴鉤捆淡鷙噁狡萎摺軀抱猛各踢朵紳梨肪誰唯樑臣均粘吞爾懲撮眨杉椒珠盞哽隘歌喔東閉團昂了幻郁嫗拗跛鑊殘哞胃同攬漓順惶貯稷郊栓饑顯哀謹襤貿舐賤橋錦冠怖嶇巢怖愕朱篷尼逐叛邊荼殼額綵孔縹透恨又'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions. 

Because our model returns logits, we need to set the `from_logits` flag. 


In [117]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 100, 3445)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       8.144802


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function. 

In [0]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 3 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=30

In [121]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
26/26 [==============================] - 14s 533ms/step - loss: 6.5264
Epoch 2/30
26/26 [==============================] - 13s 505ms/step - loss: 6.1077
Epoch 3/30
26/26 [==============================] - 13s 488ms/step - loss: 5.8168
Epoch 4/30
26/26 [==============================] - 13s 487ms/step - loss: 5.4683
Epoch 5/30
26/26 [==============================] - 13s 490ms/step - loss: 5.1483
Epoch 6/30
26/26 [==============================] - 13s 488ms/step - loss: 4.8655
Epoch 7/30
26/26 [==============================] - 13s 490ms/step - loss: 4.6563
Epoch 8/30
26/26 [==============================] - 13s 485ms/step - loss: 4.4974
Epoch 9/30
26/26 [==============================] - 13s 487ms/step - loss: 4.3742
Epoch 10/30
26/26 [==============================] - 13s 489ms/step - loss: 4.2728
Epoch 11/30
26/26 [==============================] - 13s 487ms/step - loss: 4.1845
Epoch 12/30
26/26 [==============================] - 13s 492ms/step - loss: 4.1066
Epoch 13/30
2

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built. 

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [122]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [123]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

W0410 19:16:30.613307 139812515600256 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f28237fd860>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [124]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 512)            1763840   
_________________________________________________________________
unified_lstm_3 (UnifiedLSTM) (1, None, 1024)           6295552   
_________________________________________________________________
dense_3 (Dense)              (1, None, 3445)           3531125   
Total params: 11,590,517
Trainable params: 11,590,517
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/sequences/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.6

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [128]:
print(generate_text(model, start_string=u"那姑娘道：「婢子在常熟、丹陽、無錫、嘉興等地，一共辦過七起，還有 "))


那姑娘道：「婢子在常熟、丹陽、無錫、嘉興等地，一共辦過七起，還有 
　　司馬林本相同時，只是一個人影從口中，立即刻過了去。 
　　鍾萬仇道：「不是那麼？」段譽道：「我們在下這件事，都是不信。」 
　　木婉清道：「我這小子一時便有的好人，不許這樣的人物，我可是你的心上，你這許多也不會。」 
　　段譽道：「老是在這兒子，你別生氣，你們不知如何。」 
　　諸保昆心下坦然，誰也不知。 
　　那女郎道：「你們的話可是你的事，你也不是我的。」玉虛散人道：「你不敢逃走，就是你大理。這南海派與慕容先生的武功，卻無法可能，當下了日兒。」 
　　段譽見她這鎮王夫人都是一個女子，也是不理，均是要駁不殺，也是不能讓他的話，那是不是？」 
　　那少女道：「你師叔說了你，我瞧你不識，人家改在這裡，又有什麼地道？」鍾靈道：「你是我師父。」 
　　段譽笑道：「我師父不是你的徒兒，你不知道，是不是？」 
　　鍾萬仇大驚，當即站起身來，去拉她的『段譽』」，只見到那裡一個黑布女郎，低聲道：「段公子，你父親來到他們給我，倒霉不可。」 
　　段譽見他這人影子都是一個兩根小環，每一下都是在空中，仰天之間，不知他是她的親哥哥又叫：「你……什麼？」段譽道：「你不知。你怎麼還要死。」 
　　那女郎道：「你這般確是一個好朋友。」段譽道：「你是我的身子，你們真是假藥？」段譽低聲道：「不是！」阿朱道：「那麼人？」 
　　鍾靈手中所中的神劍劍法卻仍是大丈，也不能為了。 
　　段譽道：「正是我的爹爹。 
　　青城派眾人同道：「原來姑蘇慕容兄家老爺之後，你這般性命的事，唉！」 
　　行人之中，突然間馬背上一個人影子，右目上山了一個小蛇。段譽一個不過，說道：「是！」 
　　鎮南王夫人見過彥之中的一個字：「段公子，你們便是要殺人家子。」 
　　段譽見她雙足，上了兩個人，在她身上的一件古怪，似乎不是。南海鱷神怒道：「你這是真的聰明，很像。」 
　　刀白風腮邊，忽地點頭，只見木婉清身邊一個長劍，臉色登時，已然是個人。 
　　段譽聽到這廂，心中只得一陣淒涼，心想：「這六脈神劍劍譜上有所在心上，只是不是『窮凶極惡』，這一招『金算盤』。」 
　　段譽見他這一步，也不會所說，一面也是不可。」只聽得一個女子道：「我找你的事，是不是？」段譽道：「是！」 
　　那少女道：「這個……他……她……你……我不要跟你說了。」木婉清怒道：「你的什麼一事，見了

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output. 

We will use `tf.GradientTape` to track the gradiends. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, initialize the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.



In [0]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

W0410 13:50:10.519439 139729813608320 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f14e122bac8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(target, predictions))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  
  return loss

In [0]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 12.630916595458984
Epoch 1 Loss 8.0426
Time taken for 1 epoch 15.544925689697266 sec

Epoch 2 Batch 0 Loss 7.491278171539307
Epoch 2 Loss 7.6190
Time taken for 1 epoch 13.30087947845459 sec

Epoch 3 Batch 0 Loss 7.048155307769775
Epoch 3 Loss 7.9869
Time taken for 1 epoch 13.26626181602478 sec

Epoch 4 Batch 0 Loss 7.859305381774902
Epoch 4 Loss 9.2736
Time taken for 1 epoch 13.248638391494751 sec

Epoch 5 Batch 0 Loss 9.33044719696045
Epoch 5 Loss 9.7614
Time taken for 1 epoch 13.188672304153442 sec

Epoch 6 Batch 0 Loss 9.808359146118164
Epoch 6 Loss 9.7886
Time taken for 1 epoch 13.041417121887207 sec

Epoch 7 Batch 0 Loss 9.832276344299316
Epoch 7 Loss 9.7555
Time taken for 1 epoch 13.037736654281616 sec

Epoch 8 Batch 0 Loss 9.880566596984863
Epoch 8 Loss 9.6560
Time taken for 1 epoch 13.072481155395508 sec

Epoch 9 Batch 0 Loss 9.786015510559082
Epoch 9 Loss 9.6560
Time taken for 1 epoch 13.043611288070679 sec

Epoch 10 Batch 0 Loss 9.786002159118652
Epoch 10